# Importado de librerias

In [6]:
import os
import numpy as np
import pandas as pd

# Funciones auxiliares

In [7]:
def leer_file(file : str):
    ''' 
        file : dirección al archivo a leer (config.xyz).

    return
        num_traj : cantidad de trayectorias en el archivo
        confs : Lista con {num_traj} elementos. Cada elemento es una lista con 57 elementos,
                 la primera linea caracteristicas de la conf, y las siguientes 56 la conf propiamente.
    '''
    confs = []
    with open(file) as f:
        num_trajs = int(len(f.readlines())/58)

    with open(file) as f:
        for j in range(num_trajs):    
            traj = []
            for i in range(58):
                traj.append(f.readline())
            confs.append(traj)
    print(f'Hay {num_trajs} trayectorias leidas del archivo {file}.')
    return num_trajs, confs

def leer_conf(CONF):
    ''' 
    CONF : Lista con 57 elementos extraida de `leer_file()`.

    return
        Pt : Diccionario con configuraciones de Pt del 41 a 51
        H : Diccionario con configuraciones de H de 1 a 4
        C : Configuracion de C
        ntraj : Numero de la trayectoria.

        Las configuraciones están en listas Python con coordenadas [x,y,z]
    '''
    Pt = {}
    H = {}
    ntraj = CONF[1].split()[0]
    pt_ref = float(CONF[2].split()[3])
    for i in range(31,53):
        Pt[str(i-1)] =  leer_componentes(CONF[i])
        Pt[str(i-1)][2] = Pt[str(i-1)][2] - pt_ref
    H['1'] =    leer_componentes(CONF[53])
    H['2'] =    leer_componentes(CONF[54])
    H['3'] =    leer_componentes(CONF[55])
    H['4'] =    leer_componentes(CONF[56])
    C  =    leer_componentes(CONF[57])
    return Pt, H, C, ntraj


def leer_componentes(atomo : str):
    ''' 
    atomo : Linea de texto proveniente de un archivo config.xyz de donde se extraen las componentes cartesinas a una lista [x,y,z].
    '''
    return [float(atomo.split()[1]), float(atomo.split()[2]), float(atomo.split()[3])]

def ref_leer_componentes(atomo):
    '''     
    atomo : Linea de texto proveniente de un archivo refstructureFF.out de donde se extraen las componentes cartesinas a una lista [x,y,z].
    '''
    return  [float(atomo.split()[0]), float(atomo.split()[1]), float(atomo.split()[2])]

def calcular_CM(C, H):
    m_H = 1
    m_C = 12.011
    M = 4 * m_H + m_C
    CMx = (H['1'][0] + H['2'][0] + H['3'][0] + H['4'][0] + m_C * C[0]) / M
    CMy = (H['1'][1] + H['2'][1] + H['3'][1] + H['4'][1] + m_C * C[1]) / M
    CMz = (H['1'][2] + H['2'][2] + H['3'][2] + H['4'][2] + m_C * C[2]) / M
    return [CMx, CMy, CMz]


# Lectura de datos

## Estructura de Referencia

In [8]:
Pt_ref = {}
with open('data/refstructureFF.out') as f:
            ref = f.readlines()

for i in range(3,54):
    Pt_ref[str(i-2)] =  ref_leer_componentes(ref[i])

# Automatización de generación de archivos

In [34]:
path0 = 'data/0/1-concat/'
energias =  ['Ei0.4_outputs/','Ei0.6_outputs/','Ei0.8_outputs/','Ei1.0_outputs/','Ei1.2_outputs/','Ei1.4_outputs/']

conf_files = ['config_admol.xyz',
'config_refl.xyz',
'config_stick.xyz',
'config_stickearly.xyz',
'config_sticklate.xyz']


In [35]:
for ener in energias:
   file = path0 + ener
   print(file[18:21])

0.4
0.6
0.8
1.0
1.2
1.4


In [37]:
file[18:21]

'1.4'

In [46]:
def write_all(path,file):
    ID = path[18:21].replace('.','_')
    kind = file[18:21].replace('.','_')
    print(f'id: {ID}')
    print(f'kind: {kind}')
    if not os.path.exists(path + file):
        print(f"El archivo '{file}' no existe en la ubicación '{path}'")
        return
    num_trajs, confs = leer_file(path + file)
    num_trajs_ini, confs_ini = leer_file(path + file[:-4]+'_ini.xyz')
    with open(f'{path}0K_{ID}_CH4_cm.out', 'w') as f:
        f.write(f'ntraj,x,y,z,x_ini,y_ini,z_ini\n')
        for conf,conf_ini in zip(confs,confs_ini):
            Pt, H, C, ntraj = leer_conf(conf)
            Pt_ini, H_ini, C_ini, ntraj_ini = leer_conf(conf_ini)
            CH4_cm = calcular_CM(C,H)
            CH4_cm_ini = calcular_CM(C_ini,H_ini)
            f.write(f'{ntraj},{CH4_cm[0]},{CH4_cm[1]},{CH4_cm[2]},{CH4_cm_ini[0]},{CH4_cm_ini[1]},{CH4_cm_ini[2]}\n')

    with open(f'{path}0K_{ID}_dPt.out', 'w') as f:
        f.write(f'ntraj,nat,dx,dy,dz\n')
        for conf in confs:
            Pt, H, C, ntraj = leer_conf(conf)     
            for nat in range(31,52):
                nat = str(nat)   
                v1 = np.array(Pt[nat])
                v2 = np.array(Pt_ref[nat])
                d = v1 - v2
                f.write(f'{ntraj},{nat},{d[0]},{d[1]},{d[2]}\n')
    return

def write_all_50(path,file, name=None):
    ID =path[5:6] + '_' + path[18:21].replace('.','_')
    kind = file[7:-4]
    print(f'id: {ID}')
    print(f'kind: {kind}')
    if not os.path.exists(path + file):
        print(f"El archivo '{file}' no existe en la ubicación '{path}'")
        return
    num_trajs, confs = leer_file(path + file)
    num_trajs_ini, confs_ini = leer_file(path + file[:-4]+'_ini.xyz')
    with open(f'{path}{ID}_{kind}_CH4_cm.out', 'w') as f:
        f.write(f'ntraj,x,y,z,x_ini,y_ini,z_ini\n')
        for conf,conf_ini in zip(confs,confs_ini):
            Pt, H, C, ntraj = leer_conf(conf)
            Pt_ini, H_ini, C_ini, ntraj_ini = leer_conf(conf_ini)
            CH4_cm = calcular_CM(C,H)
            CH4_cm_ini = calcular_CM(C_ini,H_ini)
            f.write(f'{ntraj},{CH4_cm[0]},{CH4_cm[1]},{CH4_cm[2]},{CH4_cm_ini[0]},{CH4_cm_ini[1]},{CH4_cm_ini[2]}\n')

    with open(f'{path}{ID}_{kind}_dPt.out', 'w') as f:
        f.write(f'ntraj,nat,dx,dy,dz\n')
        for conf in confs:
            Pt, H, C, ntraj = leer_conf(conf)     
            for nat in range(31,52):
                nat = str(nat)   
                v1 = np.array(Pt[nat])
                v2 = np.array(Pt_ref[nat])
                d = v1 - v2
                f.write(f'{ntraj},{nat},{d[0]},{d[1]},{d[2]}\n')
    return

In [30]:
config[7:-4]

'refl'

In [45]:
path= path0+Ei
path[5:6] + 'K_' + path[18:21].replace('.','_')

'0K_1_4'

In [47]:
for Ei in energias:
    for config in conf_files:
        write_all(path0+Ei,config)

id: 0_4
kind: 
El archivo 'config_admol.xyz' no existe en la ubicación 'data/0/1-concat/Ei0.4_outputs/'
id: 0_4
kind: 
El archivo 'config_refl.xyz' no existe en la ubicación 'data/0/1-concat/Ei0.4_outputs/'
id: 0_4
kind: 
El archivo 'config_stick.xyz' no existe en la ubicación 'data/0/1-concat/Ei0.4_outputs/'
id: 0_4
kind: xyz
El archivo 'config_stickearly.xyz' no existe en la ubicación 'data/0/1-concat/Ei0.4_outputs/'
id: 0_4
kind: yz
El archivo 'config_sticklate.xyz' no existe en la ubicación 'data/0/1-concat/Ei0.4_outputs/'
id: 0_6
kind: 


Hay 5410 trayectorias leidas del archivo data/0/1-concat/Ei0.6_outputs/config_admol.xyz.
Hay 5410 trayectorias leidas del archivo data/0/1-concat/Ei0.6_outputs/config_admol_ini.xyz.
id: 0_6
kind: 
Hay 3527 trayectorias leidas del archivo data/0/1-concat/Ei0.6_outputs/config_refl.xyz.
Hay 3527 trayectorias leidas del archivo data/0/1-concat/Ei0.6_outputs/config_refl_ini.xyz.
id: 0_6
kind: 
Hay 52 trayectorias leidas del archivo data/0/1-concat/Ei0.6_outputs/config_stick.xyz.
Hay 52 trayectorias leidas del archivo data/0/1-concat/Ei0.6_outputs/config_stick_ini.xyz.
id: 0_6
kind: xyz
El archivo 'config_stickearly.xyz' no existe en la ubicación 'data/0/1-concat/Ei0.6_outputs/'
id: 0_6
kind: yz
Hay 11 trayectorias leidas del archivo data/0/1-concat/Ei0.6_outputs/config_sticklate.xyz.
Hay 11 trayectorias leidas del archivo data/0/1-concat/Ei0.6_outputs/config_sticklate_ini.xyz.
id: 0_8
kind: 
El archivo 'config_admol.xyz' no existe en la ubicación 'data/0/1-concat/Ei0.8_outputs/'
id: 0_8
kin

In [14]:
# path300[8:12]
# path300[21:24]

In [48]:
for Ei in energias:
    for config in conf_files:
        write_all_50(path0+Ei,config)

id: 0_0_4
kind: admol
El archivo 'config_admol.xyz' no existe en la ubicación 'data/0/1-concat/Ei0.4_outputs/'
id: 0_0_4
kind: refl
El archivo 'config_refl.xyz' no existe en la ubicación 'data/0/1-concat/Ei0.4_outputs/'
id: 0_0_4
kind: stick
El archivo 'config_stick.xyz' no existe en la ubicación 'data/0/1-concat/Ei0.4_outputs/'
id: 0_0_4
kind: stickearly
El archivo 'config_stickearly.xyz' no existe en la ubicación 'data/0/1-concat/Ei0.4_outputs/'
id: 0_0_4
kind: sticklate
El archivo 'config_sticklate.xyz' no existe en la ubicación 'data/0/1-concat/Ei0.4_outputs/'
id: 0_0_6
kind: admol


Hay 5410 trayectorias leidas del archivo data/0/1-concat/Ei0.6_outputs/config_admol.xyz.
Hay 5410 trayectorias leidas del archivo data/0/1-concat/Ei0.6_outputs/config_admol_ini.xyz.
id: 0_0_6
kind: refl
Hay 3527 trayectorias leidas del archivo data/0/1-concat/Ei0.6_outputs/config_refl.xyz.
Hay 3527 trayectorias leidas del archivo data/0/1-concat/Ei0.6_outputs/config_refl_ini.xyz.
id: 0_0_6
kind: stick
Hay 52 trayectorias leidas del archivo data/0/1-concat/Ei0.6_outputs/config_stick.xyz.
Hay 52 trayectorias leidas del archivo data/0/1-concat/Ei0.6_outputs/config_stick_ini.xyz.
id: 0_0_6
kind: stickearly
El archivo 'config_stickearly.xyz' no existe en la ubicación 'data/0/1-concat/Ei0.6_outputs/'
id: 0_0_6
kind: sticklate
Hay 11 trayectorias leidas del archivo data/0/1-concat/Ei0.6_outputs/config_sticklate.xyz.
Hay 11 trayectorias leidas del archivo data/0/1-concat/Ei0.6_outputs/config_sticklate_ini.xyz.
id: 0_0_8
kind: admol
El archivo 'config_admol.xyz' no existe en la ubicación 'data/

In [ ]:
# for Ei in energias:
#     for config in conf_files:
#         write_all(path550+Ei,config)